In [19]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

import os
import numpy as np
from progressbar import progressbar as pbar

from omegaconf import OmegaConf
import hydra

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


check that neighbor files are a subset of original chip IDs

In [2]:
df = pd.read_parquet('/opt/data/california-naip-chips/california-naip-chips-100k.parquet')
original_chip_IDs = df['original_chip_id']

In [3]:
folder = "/opt/data/california-naip-chips/california-naip-chips-100k-neighbours"
files = pd.Series([f.removesuffix('.parquet') for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))])
files.isin(original_chip_IDs).all()

True

In [4]:
folder = "/opt/data/california-naip-chips/california-naip-chips-100k-neighbours/npy"
files = pd.Series([f.removesuffix('.npy') for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))])
files.isin(original_chip_IDs).all()

True

---

load the dataloader

In [8]:
conf = OmegaConf.load("../../configs/naip-multilabel.yaml")
conf.dataloader.neighbor_embeddings_folder = "/opt/data/california-naip-chips/california-naip-chips-100k-neighbours/npy"
conf.dataloader.batch_size = 16
dl = hydra.utils.instantiate(conf.dataloader)

2024-06-03 21:59:01.002 | INFO     | earthtext.datamodules.components.chipmultilabel:__init__:98 - using embeddings found in metadata file
2024-06-03 21:59:01.004 | INFO     | earthtext.datamodules.components.chipmultilabel:__init__:111 - read train split with 72268 chip files (out of 72268)
2024-06-03 21:59:01.004 | INFO     | earthtext.datamodules.components.chipmultilabel:__init__:115 - removing chip IDs with no associated neighbors .npy files
2024-06-03 21:59:01.447 | INFO     | earthtext.datamodules.components.chipmultilabel:__init__:123 - max cache size is -1
2024-06-03 21:59:02.923 | INFO     | earthtext.datamodules.components.chipmultilabel:__init__:98 - using embeddings found in metadata file
2024-06-03 21:59:02.925 | INFO     | earthtext.datamodules.components.chipmultilabel:__init__:111 - read val split with 14770 chip files (out of 14770)
2024-06-03 21:59:02.926 | INFO     | earthtext.datamodules.components.chipmultilabel:__init__:115 - removing chip IDs with no associated 

In [12]:
dl.train_dataset[0]['embedding'].shape

(17, 17, 768)

---

test the dataloader

In [15]:
dltrain = dl.train_dataloader()

In [16]:
batch = next(iter(dltrain))

In [18]:
batch['embedding'].shape

torch.Size([16, 17, 17, 768])